In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\ocmodels03dev\\Desktop\\Law-Chatbot\\research'

In [4]:
%cd ..

c:\Users\ocmodels03dev\Desktop\Law-Chatbot


In [5]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents 

In [7]:
extracted_data = load_pdf_file(data='data/')

In [9]:
# extracted_data

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 2442


In [13]:
# text_chunks

In [16]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

c:\Users\ocmodels03dev\Desktop\Law-Chatbot\lawbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ocmodels03dev\Desktop\Law-Chatbot\lawbot\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ocmodels03dev\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run 

In [20]:
query_result = embeddings.embed_query("Hello World")
print("Length of query is", len(query_result), "dimensional")

Length of query is 384 dimensional


In [22]:
# query_result

In [48]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [49]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [50]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [36]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'lawbot'

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

print("Index created successfully!")

Error while installing plugin inference: property 'inference' of 'Pinecone' object has no setter
Traceback (most recent call last):
  File "c:\Users\ocmodels03dev\Desktop\Law-Chatbot\lawbot\Lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'Pinecone' object has no setter


Index created successfully!


In [37]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings
)

Error while installing plugin inference: property 'inference' of 'Pinecone' object has no setter
Traceback (most recent call last):
  File "c:\Users\ocmodels03dev\Desktop\Law-Chatbot\lawbot\Lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'Pinecone' object has no setter


In [38]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

Error while installing plugin inference: property 'inference' of 'Pinecone' object has no setter
Traceback (most recent call last):
  File "c:\Users\ocmodels03dev\Desktop\Law-Chatbot\lawbot\Lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'Pinecone' object has no setter


In [39]:
docsearch

In [40]:
retriver = docsearch.as_retriever(search_type = 'similarity', search_kwargs = {"k":3})

In [41]:
retriver.invoke("What is tort?")

[Document(id='1a4a4a12-bd5f-4eca-8756-7c137a3243ae', metadata={'creationdate': '2025-03-07T17:09:11+00:00', 'creator': 'LibreTexts (libretexts.org)', 'moddate': '2025-03-07T17:09:10+00:00', 'page': 204.0, 'page_label': '205', 'producer': 'LibreTexts nodePrint', 'source': 'data\\law.pdf', 'title': 'Law 101: Fundamentals of the Law (Lumen)', 'total_pages': 316.0}, page_content='8 . 2 . 1 h t t p s : / / b i z . l i b r e t e x t s . o r g / @g o / p a g e / 6 5 6 1 4 \n8.2: GENERAL LAW OF TORTS\nWHAT IS A TORT?\nA tort is an act or omission, other than a breach of contract, which gives rise to injury or harm to another, and amounts to a civil\nwrong for which courts impose liability. In other words, a wrong has been committed and the remedy is money damages to the\nperson wronged.'),
 Document(id='2656dbca-e508-4fb9-97f3-993d100bbdf0', metadata={'creationdate': '2025-03-07T17:09:11+00:00', 'creator': 'LibreTexts (libretexts.org)', 'moddate': '2025-03-07T17:09:10+00:00', 'page': 204.0, 'p

In [60]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='llama3-8b-8192', temperature=0.5)

In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following piece of retrieved context to answer the question. "
    "If you don't know the answer say that you don't know. "
    "Use three sentences maximum and and keep the answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [62]:
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [65]:
response = rag_chain.invoke({"input": "What is Artificial Insemination?"})
print(response['answer'])

According to the provided context, Artificial Insemination is a method of conception where a child is born to a married woman by insemination, which is considered legitimate if the insemination was performed by a person authorized to practice medicine and there is written consent of the woman and her husband.


In [64]:
response = rag_chain.invoke({"input": "What is cancer?"})
print(response['answer'])

I don't know. The provided context does not mention cancer.
